In [ ]:
import numpy

import eaglecore.utils
import eaglecore.differential
import eaglecore.thresholding
import eaglecore.signal.processing

def tv(
    y: numpy.ndarray, 
    h: numpy.ndarray, 
    lamda: float, 
    sigma: float, 
    nb_iterations: int,
    tol: float = 0
) -> numpy.ndarray:
    """Total Variation Regularization with Split Bregman
    """
    
    lap_diag = eaglecore.utils.fourier_diagonalization(
        kernel = lasp.filters.linear.laplacian(),
        shape_out = y.shape
    )

    h_diag = eaglecore.utils.fourier_diagonalization(
        kernel=h,
        shape_out=y.shape
    )

    h2_diag = numpy.abs(h_diag)**2


    cst1 = h2_diag + sigma * lap_diag
    cst2 = numpy.conj(h_diag) * numpy.fft.fft2(y)
   

    # INitialization
    u = numpy.copy(y) 
    d_x=numpy.zeros_like(y)
    d_y=numpy.zeros_like(y)
    b_x=numpy.zeros_like(y)
    b_y=numpy.zeros_like(y)

    for i in range(1, nb_iterations+1):

        a = sigma * (
            eaglecore.differential.dxT(d_x-b_x)
            + eaglecore.differential.dyT(d_y-b_y)
        )

        b = numpy.fft.fft2(a) + cst2

        u0 = numpy.copy(u)
        
        u = numpy.real(numpy.fft.ifft2(b / cst1))

        err = numpy.linalg.norm(u-u0, 'fro') \
            / numpy.linalg.norm(u, 'fro')

        if i%10 == 0:
            print('Iterations: {} ! \t error is: {}'.format(i, err))

        if err <= tol:
            break

        u_dx = eaglecore.differential.dx(u)
        u_dy = eaglecore.differential.dy(u)

        d_x, d_y = eaglecore.thresholding.multidimensional_soft(
            numpy.array([u_dx+b_x, u_dy+b_y]),
            lamda/sigma
        )

        b_x += (u_dx-d_x)
        b_y += (u_dy-d_y)

    u = eaglecore.signal.processing.normalize(
        signal = u,
        new_min = 0.0,
        new_max = 1.0
    )

    return u

In [ ]:
import numpy
import numpy.fft
import typing

import eaglecore.thresholding

def create_total_variation_step(
    oper_dx: typing.Callable[[numpy.ndarray], numpy.ndarray], 
    oper_dy: typing.Callable[[numpy.ndarray], numpy.ndarray], 
    oper_dxT: typing.Callable[[numpy.ndarray], numpy.ndarray], 
    oper_dyT: typing.Callable[[numpy.ndarray], numpy.ndarray], 
    lamda: float, 
    sigma: float, 
    cst1: numpy.ndarray, 
    cst2: numpy.ndarray
) -> typing.Callable[[numpy.ndarray, numpy.ndarray, numpy.ndarray, numpy.ndarray], numpy.ndarray]:
    
    def total_variation_step(
        d_x: numpy.ndarray, 
        d_y: numpy.ndarray, 
        b_x: numpy.ndarray, 
        b_y: numpy.ndarray 
    ) -> typing.Tuple[numpy.ndarray, numpy.ndarray, numpy.ndarray, numpy.ndarray]:
    
        a = sigma * ( oper_dxT(d_x-b_x) + oper_dyT(d_y-b_y) )

        b = numpy.fft.fft2(a) + cst2
        
        u = numpy.real(numpy.fft.ifft2(b / cst1))

        u_dx = oper_dx(u)
        u_dy = oper_dy(u)

        d_x, d_y = eaglecore.thresholding.multidimensional_soft(
            numpy.array([u_dx + b_x, u_dy + b_y]),
            lamda / sigma
        )

        b_x = b_x + (u_dx - d_x)
        b_y = b_y + (u_dy - d_y)
        
        return u, d_x, d_y, b_x, b_y
    
    return total_variation_step
        

In [ ]:
import eaglecore.utils
import eaglecore.differential
import eaglecore.thresholding
import eaglecore.signal.processing
import eaglecore.filters.linear

def tv(
    y: numpy.ndarray, 
    h: numpy.ndarray, 
    lamda: float, 
    sigma: float, 
    nb_iterations: int,
    tol: float = 0.0
) -> numpy.ndarray:
    """Total Variation Regularization with Split Bregman
    """
    
    lap_diag = eaglecore.utils.fourier_diagonalization(
        kernel = eaglecore.filters.linear.laplacian(),
        shape_out = y.shape
    )

    h_diag = eaglecore.utils.fourier_diagonalization(
        kernel=h,
        shape_out=y.shape
    )

    h2_diag = numpy.abs(h_diag)**2
    
    cst1 = h2_diag + sigma * lap_diag
    cst2 = numpy.conj(h_diag) * numpy.fft.fftn(y)
    
    tv_step = create_total_variation_step(
        oper_dx = lambda arr : eaglecore.differential.dfc(arr, axis=0),
        oper_dy = lambda arr : eaglecore.differential.dfc(arr, axis=1),
        oper_dxT = lambda arr : eaglecore.differential.tdfc(arr, axis=0),
        oper_dyT = lambda arr : eaglecore.differential.tdfc(arr, axis=1),
        lamda = lamda,
        sigma = sigma,
        cst1 = cst1, cst2 = cst2 
    )
    
    u0 = numpy.copy(y) 
    d_x = numpy.zeros_like(y)
    d_y = numpy.zeros_like(y)
    b_x = numpy.zeros_like(y)
    b_y = numpy.zeros_like(y)
      
    for i in range(1, nb_iterations+1):
        
        u, d_x, d_y, b_x, b_y = tv_step(d_x, d_y, b_x, b_y)
        
        err = numpy.linalg.norm(u-u0, 'fro') \
            / numpy.linalg.norm(u, 'fro')

        if i%10 == 0:
            print('Iterations: {} ! \t error is: {}'.format(i, err))

        if err <= tol:
            break
        
    

In [ ]:
TupleArray3 = typing.Tuple[numpy.ndarray, numpy.ndarray, numpy.ndarray]

def create_total_variation_step_v2(
    oper_dx: typing.Callable[[numpy.ndarray], numpy.ndarray], 
    oper_dy: typing.Callable[[numpy.ndarray], numpy.ndarray], 
    oper_dxT: typing.Callable[[numpy.ndarray], numpy.ndarray], 
    oper_dyT: typing.Callable[[numpy.ndarray], numpy.ndarray], 
    lamda: float, 
    sigma: float, 
    cst1: numpy.ndarray, 
    cst2: numpy.ndarray
) -> typing.Callable[[numpy.ndarray, numpy.ndarray], TupleArray3] :
    
    def total_variation_step(
        d: numpy.ndarray, # (2, N, M)
        b: numpy.ndarray, # (2, N, M)
    ) -> TupleArray3:
    
        calc1 = sigma * ( oper_dxT(d[0]-b[0]) + oper_dyT(d[1]-b[1]) ) # (N, M)

        calc2 = numpy.fft.fftn(calc1) + cst2 # (N, M)
        
        u = numpy.real(numpy.fft.ifftn(calc2 / cst1)) # (N, M)

        grad_u = numpy.array([ oper_dx(u), oper_dy(u) ]) # (2, N, M)

        d_next = eaglecore.thresholding.multidimensional_soft(
            grad_u + b,
            lamda / sigma
        ) # (2, N, M)

        b_next = b + (grad_u - d_next) # (2, N, M)
        
        return u, d_next, b_next
    
    return total_variation_step

In [ ]:
import eaglecore.utils
import eaglecore.differential
import eaglecore.thresholding
import eaglecore.signal.processing
import eaglecore.filters.linear

def tv(
    y: numpy.ndarray, 
    h: numpy.ndarray, 
    lamda: float, 
    sigma: float, 
    nb_iterations: int,
    tol: float = 0.0
) -> numpy.ndarray:
    """Total Variation Regularization with Split Bregman
    """
    
    lap_diag = eaglecore.utils.fourier_diagonalization(
        kernel = eaglecore.filters.linear.laplacian(),
        shape_out = y.shape
    )

    h_diag = eaglecore.utils.fourier_diagonalization(
        kernel=h,
        shape_out=y.shape
    )

    h2_diag = numpy.abs(h_diag)**2
    
    cst1 = h2_diag + sigma * lap_diag
    cst2 = numpy.conj(h_diag) * numpy.fft.fftn(y)
    
    tv_step = create_total_variation_step_v2(
        oper_dx = lambda arr : eaglecore.differential.dfc(arr, axis=0),
        oper_dy = lambda arr : eaglecore.differential.dfc(arr, axis=1),
        oper_dxT = lambda arr : eaglecore.differential.tdfc(arr, axis=0),
        oper_dyT = lambda arr : eaglecore.differential.tdfc(arr, axis=1),
        lamda = lamda,
        sigma = sigma,
        cst1 = cst1, cst2 = cst2 
    )
    
    u0 = numpy.copy(y) 
    d = numpy.zeros(shape = (2, y.shape[0], y.shape[1]))
    b = numpy.zeros(shape = (2, y.shape[0], y.shape[1]))
      
    for i in range(1, nb_iterations+1):
        
        u, d, b = tv_step(d, b)
        
        err = numpy.linalg.norm(u-u0, 'fro') \
            / numpy.linalg.norm(u, 'fro')

        if i%10 == 0:
            print('Iterations: {} ! \t error is: {}'.format(i, err))

        if err <= tol:
            break

In [12]:
import numpy
import numpy.fft


arr = numpy.random.rand(2, 4, 4)
arr

arr_fft_ = numpy.array(
    [numpy.fft.fft2(arr[0]), numpy.fft.fft2(arr[1])]
)
arr_fft_

arr_fft = numpy.fft.fft2(arr, axes=[1, 2])
arr_fft

numpy.fft.fftn(arr, axes = [1, 2])

array([[[ 7.08279084+0.j        ,  0.43138919-1.1758553j ,
          0.8446633 +0.j        ,  0.43138919+1.1758553j ],
        [ 0.12303442-1.0233237j , -0.43570235+0.10248436j,
          1.0492727 -0.73071561j,  0.62312533+0.93014696j],
        [ 1.84549884+0.j        ,  0.85553356-0.72168894j,
         -0.55092788+0.j        ,  0.85553356+0.72168894j],
        [ 0.12303442+1.0233237j ,  0.62312533-0.93014696j,
          1.0492727 +0.73071561j, -0.43570235-0.10248436j]],

       [[ 8.54692116+0.j        , -0.52058224-0.8556335j ,
          0.74436953+0.j        , -0.52058224+0.8556335j ],
        [-1.06042451+1.06288015j, -0.2455616 +0.33623352j,
         -0.11632324+1.01479391j,  0.05700076-0.53954572j],
        [-0.14718493+0.j        ,  0.08751988+0.36348709j,
          1.97991891+0.j        ,  0.08751988-0.36348709j],
        [-1.06042451-1.06288015j,  0.05700076+0.53954572j,
         -0.11632324-1.01479391j, -0.2455616 -0.33623352j]]])

In [1]:
def f(x: float) -> float:
    return x**2

def df(x: float) -> float:
    return 2*x

def minimize_f(start: float, nb_epochs: int, learning_rate: float, verbose: int) -> float:
    
    x = start
    for no_epoch in range(0, nb_epochs):
        x = x - learning_rate * df(x)
        if(nb_epochs % verbose == 0):
            print('Epoch {} :'.format(no_epoch), x, f(x))
    
    return f(x)

minimize_f(start = 3, nb_epochs = 1000, learning_rate = 0.1, verbose=100)

Iteration 0 : 2.4 5.76
Iteration 100 : 4.888886343202771e-10 2.3901209676754565e-19
Iteration 200 : 9.95883736531439e-20 9.917844166878207e-39
Iteration 300 : 2.0286509995609565e-29 4.115424878019668e-58
Iteration 400 : 4.132435069532589e-39 1.7077019603902814e-77
Iteration 500 : 8.417918906504187e-49 7.086135871648064e-97
Iteration 600 : 1.7147603658415285e-58 2.940403112260973e-116
Iteration 700 : 3.4930285560116794e-68 1.2201248493113037e-135
Iteration 800 : 7.11542483495949e-78 5.062927058195828e-155
Iteration 900 : 1.4494376375716342e-87 2.10086946520924e-174


1.3621235240738185e-193

In [7]:
def f(x: float) -> float:
    return 1 / x

def df(x: float) -> float:
    return -1 / (x**2)

def minimize_f(start: float, nb_iters: int, learning_rate: float, verbose: int) -> float:
    
    x = start
    for i in range(0, nb_iters):
        x = x - learning_rate * df(x)
        if(i % verbose == 0):
            print('Iteration {} :'.format(i), x, f(x))
    
    return x, f(x)

minimize_f(start = 3, nb_iters = 1000, learning_rate = 1_000_000, verbose=100)

Iteration 0 : 111114.11111111111 8.999757006560823e-06
Iteration 100 : 111114.11921067319 8.999756350531769e-06
Iteration 200 : 111114.12731023415 8.9997556945029e-06
Iteration 300 : 111114.13540979379 8.999755038474234e-06
Iteration 400 : 111114.14350935219 8.999754382445764e-06
Iteration 500 : 111114.15160890947 8.999753726417481e-06
Iteration 600 : 111114.15970846562 8.999753070389384e-06
Iteration 700 : 111114.16780802066 8.999752414361474e-06
Iteration 800 : 111114.17590757448 8.999751758333758e-06
Iteration 900 : 111114.18400712695 8.999751102306248e-06


(111114.1920256828, 8.999750452839194e-06)